In [1]:
from pandas_datareader import data
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
import numpy as np
%matplotlib inline

In [2]:
FB = pd.read_csv('SPX.csv')

In [3]:
def ichimoku_cloud(df,w9,w26,w52,chikou26):
    #crypto settings are 20, 60, 120, 30 see Josh video for confirmation
    #https://www.youtube.com/watch?v=5x0r-qcGoQQ&t=1s
    
    
    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    df['period9_high'] = pd.Series.rolling(df['High'], w9).max()
    df['period9_low'] = pd.Series.rolling(df['Low'], w9).min()
    df['tenkan_sen'] = (df['period9_high'] + df['period9_low']) / 2
    
    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    df['period26_high'] = pd.Series.rolling(df['High'], w26).max()
    df['period26_low'] = pd.Series.rolling(df['Low'], w26).min()
    df['kijun_sen'] = (df['period26_high'] + df['period26_low']) / 2
    
    df['period52_high'] = pd.Series.rolling(df['High'], w52).max()
    df['period52_low'] = pd.Series.rolling(df['Low'], w52).min()
    df['senkou_span_a'] = (df['tenkan_sen'] + df['kijun_sen']) / 2
    df['senkou_span_b'] = (df['period52_high'] + df['period52_low'] ) /2 
    
    df ['chikou_span'] = df['Close'].shift(chikou26) 
    return df


In [4]:
ichimoku_cloud(FB,9,26,52,26)
FB.dropna(inplace = True)

In [ ]:

def returns_df(df):
    df['tk position'] = None
    df['tk senkou b position'] = None
    df['tk senkou a position'] = None
    
    for row in range(len(df)):
        if df['tenkan_sen'].iloc[row] > df['kijun_sen'].iloc[row]:
            df['tk position'].iloc[row] = 1
        else:
            df['tk position'].iloc[row] = -1

    for row in range(len(df)):
        if df['Adj Close'].iloc[row] > df['senkou_span_b'].iloc[row]:
            df['tk senkou b position'].iloc[row] = 1
        else:
            df['tk senkou b position'].iloc[row] = -1

    for row in range(len(df)):
        if df['Adj Close'].iloc[row] > df['senkou_span_a'].iloc[row]:
            df['tk senkou a position'].iloc[row] = 1
        else:
            df['tk senkou a position'].iloc[row] = -1
            
    df['tk position'].fillna(method = 'ffill', inplace = True)
    df['tk senkou b position'].fillna(method = 'ffill', inplace = True)
    df['tk senkou a position'].fillna(method = 'ffill', inplace = True)
    df['market return'] = np.log(df['Adj Close'] / df['Adj Close'].shift(1))
    df['strategy return tk'] = df['tk position'] * df['market return']
    df['strategy return senkou b'] = df['tk senkou b position'] * df['market return']
    df['strategy return senkou a'] = df['tk senkou a position'] * df['market return']
    
    df['strategy return tk'].cumsum().plot(grid = True, figsize = (20,5), color = 'g')
    df['strategy return senkou b'].cumsum().plot(grid = True, figsize = (20,5), color = 'b')
    df['strategy return senkou a'].cumsum().plot(grid = True, figsize = (20,5), color = 'r')
    df['market return'].cumsum().plot(grid = True, figsize = (20,5), color = 'm')
    
            
    return df

In [ ]:
FB.dropna(inplace = True)
returns_df(FB)

C:\Users\xexx\Anaconda3\lib\site-packages\pandas\core\indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
def risk_measures(df):
    pass
#sharpe, calmar, max drawdown

In [ ]:
FB